In [2]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Model
from azureml.core.conda_dependencies import CondaDependencies

# working with script argument 

In [3]:
ws = Workspace.from_config()

# Here we are using the environment created in previous notebook. 
# We can retrive the environment because it was registered in the workspace. 
myenv = Environment.get(workspace=ws, 
                        name='environemnt-002')

In [4]:
# We train a random forest model in the script. 
# We can provide the hyperparameter of the model as an argument in the 
# script_config object which is parsed in the script. 
script_config = ScriptRunConfig(source_directory='.', 
                                script='experiment_3.py', 
                                arguments=['--num_tree', 5], 
                                environment=myenv)

In [11]:
# This part of the code is same as before. 
experiment = Experiment(workspace=ws, name='exp-004')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

RunId: exp-004_1633451654_77e489c7
Web View: https://ml.azure.com/runs/exp-004_1633451654_77e489c7?wsid=/subscriptions/038a8790-7ab1-483b-abba-30f101e8dcce/resourcegroups/aml-resources-mstutorial/workspaces/aml-mstutorial&tid=68fda48c-5b34-479d-91f9-034da6f0efe3

Streaming azureml-logs/70_driver_log.txt

[2021-10-05T16:34:17.547129] Entering context manager injector.
[2021-10-05T16:34:18.432186] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['experiment_3.py', '--num_tree', '5'])
Script type = None
[2021-10-05T16:34:18.432186] Entering Run History Context Manager.
[2021-10-05T16:34:19.673239] Current directory: C:\Users\rosha\AppData\Local\Temp\azureml_runs\exp-004_1633451654_77e489c7
[2021-10-05T16:34:19.673239] Preparing to call script [experiment_3.py] with arguments:['--num_tree', '5']
[2021-10-05T16:34

{'runId': 'exp-004_1633451654_77e489c7',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-10-05T16:34:16.482637Z',
 'endTimeUtc': '2021-10-05T16:34:47.108696Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '0b71f75d-9f59-4263-a22f-9159992f8370',
  'azureml.git.repository_uri': 'https://github.com/roshankoirala/MLOps.git',
  'mlflow.source.git.repoURL': 'https://github.com/roshankoirala/MLOps.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'experiment_3.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_tree', '5'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCou

# Downloading model files 

In [6]:
# We can see the files logged during the experiment. 
# First three files are produced by default in azure. 
# The last file is the serialized model we logged for this run 
# through the script. 
for file in run.get_file_names():
    print(file)

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/15744_azureml.log
outputs/model.pkl


In [7]:
# We can download the model from the cloud to the local computer. 
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')

# Register a model 

In addition to `Experiments` the azure ml has another very important class called `Models` where we can register the models which we can later use for the production. Here we present two ways to register a model. 
- We may train the model locally. Then save the model locally as a `.pkl` or similar file. We can upload the register the model in the cloud. 
- We may run the experiment and have model as logged file from the experiment already present in the cloud. We can register the logged model directly as well. 

In [13]:
# From local file 
model = Model.register(workspace=ws, 
                       model_name='classification_model', 
                       model_path='model.pkl',  # local path 
                       description='First registered model', 
                       tags={'model-class':'random-forest', 
                             'preprocessing':'zero-feature-engineering'}, 
                       model_framework=Model.Framework.SCIKITLEARN)

Registering model classification_model


In [14]:
# from cloud output 
run.register_model(model_name='classification_model', 
                   model_path='outputs/model.pkl', 
                   description='Registered from run in cloud', 
                   tags={'model-class':'random-forest', 
                             'preprocessing':'zero-feature-engineering'}, 
                   model_framework=Model.Framework.SCIKITLEARN)

Model(workspace=Workspace.create(name='aml-mstutorial', subscription_id='038a8790-7ab1-483b-abba-30f101e8dcce', resource_group='aml-resources-mstutorial'), name=classification_model, id=classification_model:6, version=6, tags={'model-class': 'random-forest', 'preprocessing': 'zero-feature-engineering'}, properties={})

In [15]:
# Model with the same name are saved as a separate version of the same model by default. 
# We can list the name and version of the model in the following way. 
for model in Model.list(ws):
    print(model.name, 'Version:', model.version)

classification_model Version: 6
classification_model Version: 5
classification_model Version: 4
classification_model Version: 3
classification_model Version: 2
classification_model Version: 1
